# Fazit

Das Ziel der Studienarbeit, eine künstliche Intelligenz für das Brettspiel Mühle zu entwickeln, wurde erreicht. Es wurden zwei Algorithmen mit verschiedenen Verbesserungen implementiert, die ein interessantes Spiel gegen Menschen ermöglichen.

## Bewertung

Um eine möglichst objektive Bewertung der Algorithmen durchzuführen, sollen in diesem Kapitel die Algorithmen *Minimax* und *α-β-Pruning*, sowie mehrere Heuristiken ausprobiert werden. Die dazu benötigten Implementierungen wurden bereits im Kapitel *Turnier* vorgenommen.

In [ ]:
import os.path, struct
css = ""
if os.path.isfile("style.html"):
    from IPython.core.display import HTML
    with open("style.html", "r") as file:
        css = file.read()
HTML(css)

Das zuvor beschriebene Modul *Turnier* muss geladen werden. Durch das Laden des Moduls werden bereits alle anderen nötigen Module wie *Minimax*, *α-β-Pruning* oder die *Game*-Definition geladen.

In [ ]:
%run nmm-tournament.ipynb

### Minimax vs. α-β-Pruning
In dem ersten Experiment sollen die Algorithmen *Minimax* und *α-β-Pruning* gegeneinander antreten. Beide verwenden die selbe Heuristik, welche zufällig ausgewählt wurde. Insgesamt werden zwei Runden à vier Spiele gespielt. Jeder Algorithmus beginnt einmal als der *weiße* Spieler.

* *α-β-Pruning* betrachtet 25.000 Zustande pro Zug. Dies dauert in jeder Phase des Spiels ca. 10 Sekunden.
* *Minimax* hingegen durchsucht einen Baum mit der Tiefe drei und schaut somit drei Spielzüge in die Zukunft. Dies dauert zu beginn des Spiels ca. 35 Sekunden, in der zweiten Phase hingegen ca. 1-2 Sekunden.

Zu erwarten ist, dass *α-β-Pruning* besser als *Minimax* abschneidet, da es die Möglichkeit besitzt Teilbäume abzuschneiden, welche nicht mehr in Frage kommen würden. Außerdem kann *α-β-Pruning* in der zweiten Phase des Spiels aufgrund der *iterativen Tiefensuche* eine größere Rekursionstiefe erreichen.

In [ ]:
Tournament(
    [
        lambda: Minimax(
            weights    = HeuristicWeights(stones=1, stash=1, mills=4, possible_mills=2),
            limit      = 3
        ),
        lambda: AlphaBetaPruning(
            weights    = HeuristicWeights(stones=1, stash=1, mills=4, possible_mills=2),
            max_states = 25_000
        )
    ],
    instances_per_round = 4,
    name                = "mm-vs-ab"
).play()

![](images/nmm-mm-vs-ab.png)

Die vorherige Vermutung lässt sich mit diesem Ergebnis bestätigen: *α-β-Pruning* schneidet besser ab als *Minimax*. Dennoch ist es überraschend, dass zwei der Spiele im Remis enden und weitere zwei Spiele sogar von Minimax gewonnen werden.

Zu dem Gesamtsieg von *α-β-Pruning* hat einerseits der Algorithmus selbst beigetragen, da dadurch weite Teile des Suchbaumes übersprungen werden konnten. Die Verwendung der *iterativen Tiefensuche* hat mit einer festen Suchgröße dazu beigetragen, dass bei gleichbleibender Rechenzeit dynamisch die richtige Rekursionstiefe gewählt werden konnte. Die Erkennung von symmetrischen Zuständen hilft besonders in den ersten Zügen eine große Rekursionstiefe zu erreichen, da hier viele Symmetrien auftauchen. Auch in der letzten fliegenden Phase können ein paar Rechnung damit eingespaart werden.

### Heuristik vs. Heuristik (α-β-Pruning)
In dem zweiten Experiment soll unter Verwendung des α-β-Pruning Algorithmus herausgefunden werden, welche Heuristik am besten geeignet ist.
Die Anzahl der möglichen Heuristiken ist unendlich groß, da jeder der vier Parameter eine reelle Zahl ist. Aus diesem Grund soll nur ein kleines Experiment durchgeführt werden.
Es treten sechs Konfigurationen in 3 Spielen pro Runde an. Dadurch ergibt sich eine Rundenanzahl von 30.

Bei diesem Experiment soll die Wichtigkeit der Parameter festgestellt werden, indem alle möglichen Heuristiken mit den Permutationen der Zahlen $1, 2, 3$ gegeneinander antreten. Die Parameter `stones` und `stash` erhalten jedoch immer den gleichen Wert, da sich während der Entwicklung gezeigt hat, dass die Algorithmen besonders in der ersten Phase falsche Entscheidungen treffen, sollten sich diese Parameter unterscheiden. Als positiver Nebeneffekt verringert sich die Größe des Experimentes.

In [ ]:
Tournament(
    [
        lambda: AlphaBetaPruning(weights=HeuristicWeights(stones=1, stash=1, mills=2, possible_mills=3)),
        lambda: AlphaBetaPruning(weights=HeuristicWeights(stones=2, stash=2, mills=1, possible_mills=3)),
        lambda: AlphaBetaPruning(weights=HeuristicWeights(stones=3, stash=3, mills=2, possible_mills=1)),
        lambda: AlphaBetaPruning(weights=HeuristicWeights(stones=1, stash=1, mills=3, possible_mills=2)),
        lambda: AlphaBetaPruning(weights=HeuristicWeights(stones=2, stash=2, mills=3, possible_mills=1)),
        lambda: AlphaBetaPruning(weights=HeuristicWeights(stones=3, stash=3, mills=1, possible_mills=2)),
    ],
    instances_per_round = 3,
    name                = "hr-vs-hr",
).play()

Die Rechenzeit für dieses Experiment betrug ca. 12 Stunden. Dabei reichte zwischenzeitlich der Arbeitsspeicher von 32GB nicht aus und das Experiment brach kurzzeitig ab.

Die aggregierten Ergebnisse sind in der folgenden Tabelle dargstellt. Die detaillierten Ergebnisse sind in den Dateien `round-hr-vs-hr-{1-30}.txt` zu finden. Die Spalten stellen die *weißen* Spieler dar und die Zeilen die *schwarzen* Spieler. Jede Heuristik hat einmal als *weiß* und einmal als *schwarz* gegen jede andere Heuristik gespielt. Gegen sich selbst wurde nicht gespielt (siehe Diagonale ohne Daten).

Der Bezeichner für jede Heurisik ist als Tupel aufgeschrieben, die folgender Definition entspricht:
$$\langle stones, stash, mills, possible\_mills \rangle$$
Eine Zelle in der Tabelle stellt eine Runde dar und zählt die Anzahl der Spiele in drei Kategorien:
$$Weiß\ gewinnt\ |\ Remis\ |\ Schwarz\ gewinnt$$


![](images/nmm-hr-vs-hr-rounds.png)

Aggregiert ergibt sich aus den Rundenergebnissen folgende Tabelle. Alle Spiele der Heuristiken wurden aufgeschlüsselt in
$$Gewonnen\ |\ Remis\ |\ Verloren $$
Für die Punkteberechnung wurde jedes Gewinnen mit $+1$ und jedes Verlieren mit $-1$ bewertet. Ein Remis hat eine Wertung von $0$.

![](images/nmm-hr-vs-hr-total.png)

Als klare Verlierer sind die Heuristiken zu bewerten, welche den Parameter $possible\_mills$ größer als die anderen Parameter gewählt haben. Beide haben mehr als die Hälfte der 30 gespielten Spiele verloren und haben somit auch eine sehr negative Punktwertung.

Mit einem Punkt Vorsprung gewinnt die Heuristik $\langle 3,3,2,1 \rangle$ in der Gesamtwertung. Diese gewichtet $stones$ und $stash$ höher als $mills$, welche wiederum höher als $possible\_mills$ gewichtet werden. Der zweite Platz $\langle 2,2,3,1 \rangle$ verliert nur mit einem Punkt. Auch im direkten Vergleich gewann der erste Platz nur einmal öfter gegen den zweiten Platz.

Da diese Werte sehr nah aneinander liegen, können diese kleinen Abweichungen auch durch Zufall entstanden sein. Bei gleich bewerteten Zügen wählt der Algorithmus einen Spielzug zufällig aus.

## Verbesserungsmöglichkeiten

Die Implementierung benötig sehr viel Arbeitsspeicher, was jedoch beim normalen Spielen kein Problem darstellen sollte. Erst wenn zu Testzwecken mehrere Spiele gleichzeitig gespielt werden, macht sich der große Arbeitsspeicherverbrauch bemerkbar. Durch die Verwendung einer Bitmaske könnte der Arbeitsspeicher um ein Vielfaches verringert werden. Diese Änderungen wären jedoch sehr gravierend gewesen.

Die Rechenzeit der Algorithmen ist sehr hoch. Dies hängt stark mit dem Arbeitsspeicherverbrauch zusammen, da hierdurch mehr Daten kopiert werden müssen und Berechnungen auf Grund der größeren Datenmengen länger brauchen. Eine Verringerung der Rechenzeit könnte die Algorithmen aus Sicht des Anwenders besser werden lassen, da in der gleichen Zeit mehr Zustände in einer größeren Rekursionstiefe, und damit weitere Züge in der Zukunft, betrachtet werden.

Das Lösen dieser Probleme würde eine komplette Neuimplementierung der Algorithmen nötig machen. Im gleichen Zuge könnte dann jedoch eine schnellere, kompilierte Sprache für die Implementierung gewählt werden, beispielsweise *C*, *C++* oder *Rust*.